In [1]:

from pathlib import Path
from openai import OpenAI
import os
import json
file_name='工作票1'
with open(f'./data/工作票/{file_name}/rule.txt',encoding='utf-8') as f:
    rule=f.read()
with open(f'./data/工作票/{file_name}/instance.txt',encoding='utf-8') as f:
    doc=f.read()


client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

In [2]:
question=f"""
## 任务
请检查以下文档是否符合以下合规要求，并输出json格式的结果，包括错误的项，匹配的规则，错误的原因，修改建议。每条规则只检测 规则中的【】项。如果项内容为空，则跳过检测。

##规则
{rule}

##文档
{doc}

##输出*json格式*的结果。例如
[{{
'node':'1',
 'rule':'2',
 'explain':'天数为20天，超出规则设定的15天',
'suggestion':'检查天数是否填写错误'}}
]
其中node代表错误项，rule代表匹配的规则，explain代表详细的解释。如果没有错误，则跳过此项。
"""
print(question)


## 任务
请检查以下文档是否符合以下合规要求，并输出json格式的结果，包括错误的项，匹配的规则，错误的原因，修改建议。每条规则只检测 规则中的【】项。如果项内容为空，则跳过检测。

##规则
1. 【工作班成员】。检查签名数量。

   - 如果人数小于等于5，验证签名人数少于5人。
   - 如果人数大于5人，检查 最后一个名字后是否加上“等”字，且只列出5个人的签名。

2. 【停电线路名称】。检查名称是否填写规范，要求填写：

    电压等级：必须明确写出数字和单位（如 10kV）。
    线路双重名称：由地点名称、大写字母、数字和“线”组成（如 里金555线）。

3. 【工作任务】。检查工作地点栏是否填写规范，要求包含以下内容：

    电压等级：必须明确写出数字和单位（如 10kV）。
    线路双重名称：由地点名称、大写字母、数字和“线”组成（如 里金555线）。
    起止杆号：需标明起始杆号和终止杆号，支持多种形式的杆号表示（如 96+1号杆 或直接写 2号杆），并用“至”连接起止位置。

4.  安全措施。略过。

5. 【接地线】。工作班现场装拆的接地线（个人保安线），数量是否匹配。

如果您希望我正确判断规则1，可以对规则进行更明确的描述，确保规则逻辑清晰且易于解析。以下是针对规则1的优化建议：

### 优化后的规则1：
```plaintext
1. 【工作班成员】。检查签名数量。
   - 如果人数小于等于5人，则列出所有成员姓名。
   - 如果人数大于5人，则仅填写前5个姓名，并在最后一个名字后加上“等”。例如：a，b，c，d，e等。
   

##文档
单位：工程管理部
编号：台州-三门-工程管理部-业扩项目班-2025-04-PI-003[数]

1. 工作负责人：毛俊强
班组：业扩项目班

2. 工作班成员 (不包括工作负责人)：
傅东根，李强，李林，徐贵强，彭祝化等
  			共_8_人

停电线路名称(多回线路应注明双重称号)：10kV季宅B918线，季宅B918线，10kVB918线，10kV季宅

3. 工作任务：
|工作地点(地段)或设备|  工作内容|
|-------------------------------|--------------|
|10kV里金555线96+1号杆至芹溪村2号变支

In [3]:

completion = client.chat.completions.create(
model="qwen-plus",
messages=[
    {'role': 'system', 'content': "你是一个文档校对助手，根据*规则*，判断*文档*是否填写规范。"},
    {'role': 'user', 'content': question}
    ]
)
res=completion.choices[0].message.content
print(res)


```json
[
  {
    "node": "2",
    "rule": "1",
    "explain": "工作班成员签名数量大于5人，但未列出前5个姓名并以‘等’结尾。文档中已列出5人，但未明确是否为前5人。",
    "suggestion": "确保列出的是前5位成员姓名，并在最后一个名字后加上‘等’字。例如：傅东根，李强，李林，徐贵强，彭祝化等。"
  },
  {
    "node": "3",
    "rule": "2",
    "explain": "停电线路名称填写不规范。部分名称缺少电压等级或双重名称格式不符合要求（如‘10kVB918线’和‘10kV季宅’）。",
    "suggestion": "所有停电线路名称应包含电压等级和完整的双重名称，格式为‘电压等级+地点名称+大写字母+数字+线’。例如：10kV季宅B918线。"
  },
  {
    "node": "4",
    "rule": "3",
    "explain": "工作任务的工作地点栏填写不完整。虽然包含了电压等级和线路双重名称，但未明确起止杆号。",
    "suggestion": "在工作地点栏中补充起止杆号信息，例如：10kV里金555线96+1号杆至芹溪村2号变支线2号杆。"
  },
  {
    "node": "7",
    "rule": "5",
    "explain": "接地线数量未匹配。工作班装设(或拆除)的接地线表格为空，未记录实际装设或拆除的接地线信息。",
    "suggestion": "补充工作班装设或拆除的接地线详细信息，包括线路名称、设备双重名称、装设位置、接地线编号、装拆人员及时间。"
  }
]
``` 

### 解释：
1. **规则1**：检查工作班成员签名数量。文档中列出了5人以上成员，但未明确是否为前5人且加“等”。
2. **规则2**：检查停电线路名称是否规范。文档中部分线路名称缺少电压等级或双重名称格式错误。
3. **规则3**：检查工作任务的工作地点栏是否规范。文档中未明确起止杆号。
4. **规则5**：检查接地线数量是否匹配。文档中未记录工作班装设或拆除的接地线信息。


ocr识别

json[]转.格式

In [5]:
import json
from types import SimpleNamespace

json_str = '''
{
    "user": {
        "name": "Alice",
        "age": 25
    },
    "location": "Wonderland"
}
'''

data = json.loads(json_str, object_hook=lambda d: SimpleNamespace(**d))

print(data.user.name)  # Alice
print(data.location)   # Wonderland

Alice
Wonderland
